In [1]:
import numpy as np
import cv2
import torch 
import torchvision 
from engine import train_one_epoch, evaluate
from data_reading import WindowDataset
import utils
from torch.utils.data import DataLoader

In [2]:
def get_model():
    # Initialize Model, from pytorch tutorial: https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained = True)
    num_classes = 2 # window + background
    in_features = model.roi_heads.box_predictor.cls_score.in_features # Number of input features for the classifier
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
    return model.double()

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'running on device {device}')
model = get_model()

running on device cpu


In [4]:
params = [p for p in model.parameters() if p.requires_grad]

optimizer = torch.optim.Adam(params, lr = 0.00001)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 3, gamma = 0.1)

In [5]:
train = DataLoader(WindowDataset(), batch_size=1, shuffle=True, collate_fn=utils.collate_fn)
print(next(iter(train))[0])
print(next(iter(train))[1])

(tensor([[[154., 167., 172.,  ..., 222., 221., 220.],
         [156., 165., 166.,  ..., 223., 223., 222.],
         [157., 161., 160.,  ..., 227., 227., 226.],
         ...,
         [ 92., 145., 158.,  ..., 255., 255., 255.],
         [ 91., 144., 156.,  ..., 255., 255., 255.],
         [ 89., 143., 154.,  ..., 255., 255., 255.]],

        [[192., 206., 208.,  ..., 252., 251., 250.],
         [191., 201., 200.,  ..., 253., 253., 252.],
         [191., 195., 192.,  ..., 254., 254., 253.],
         ...,
         [ 99., 152., 165.,  ..., 255., 255., 255.],
         [ 98., 151., 163.,  ..., 255., 255., 255.],
         [ 96., 150., 161.,  ..., 255., 255., 255.]],

        [[193., 205., 208.,  ..., 242., 241., 240.],
         [193., 201., 199.,  ..., 243., 243., 242.],
         [192., 196., 191.,  ..., 245., 245., 244.],
         ...,
         [ 92., 145., 158.,  ..., 255., 255., 255.],
         [ 91., 144., 156.,  ..., 255., 255., 255.],
         [ 89., 143., 154.,  ..., 255., 255., 255.]]

In [6]:
print(next(iter(train))[1][0]['labels'].dtype)

torch.int64


In [7]:
imgs, labs = next(iter(train))
model.eval()
model(imgs)

/Users/michaelyhuang/ProjectData/python/virtual-environments/ml-venv/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[{'boxes': tensor([[  0.0000,   0.0000, 128.0000,  96.0000],
          [  0.0000,  21.1592, 128.0000,  33.0713],
          [ 42.7368,   0.0000,  98.8102,  96.0000],
          [  0.0000,  59.5675, 128.0000,  72.6507],
          [ 82.0314,   0.0000, 128.0000,  96.0000],
          [100.2715,  43.2387, 128.0000,  46.3143],
          [ 66.9151,   0.0000, 128.0000,  56.3966],
          [  0.0000,  36.1535, 128.0000,  82.2693],
          [  0.0000,   0.0000,  17.3021,  47.8244],
          [  0.0000,  53.7799, 128.0000,  64.5278],
          [  0.0000,  24.9707,   3.1109,  96.0000],
          [  0.0000,  48.9071,  96.7082,  68.4073],
          [  0.0000,  91.9571,  14.6096,  96.0000],
          [  0.0000,  82.2522,  40.0084,  96.0000],
          [  0.0000,  75.5324,  21.4081,  95.8062],
          [  0.0000,   0.0000, 128.0000,  43.7453],
          [  0.0000,  69.2929, 128.0000,  92.8344],
          [  0.0000,  28.5135,  19.3651,  35.0734],
          [  4.9449,  81.0390,  11.8473,  84.5752],
   

In [8]:
num_epochs = 40

for epoch in range(num_epochs):
    # Train for one epoch, while printing every 10 iterations
    train_one_epoch(model, optimizer, train, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset

print("DONE")

Epoch: [0]  [   0/1029]  eta: 10:16:05  lr: 0.000010  loss: 45.0131 (45.0131)  loss_classifier: 3.3153 (3.3153)  loss_box_reg: 0.6038 (0.6038)  loss_objectness: 41.0407 (41.0407)  loss_rpn_box_reg: 0.0533 (0.0533)  time: 35.9234  data: 0.0017
